In [2]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
import sys
sys.path.append('../../../')
import keys as k
from time import sleep
from datetime import datetime, date
from concurrent.futures import ThreadPoolExecutor
import os
import glob

In [5]:
# Define las palabras clave relevantes para buscar los tweets
palabras_clave = ["bitcoin", "BTC", "$BTC"]

### Prueba de Extracción Exitosa No. 1

In [ ]:
for index, row in dates.iterrows():
    tweets_list = []
    naming = row['start_date']
    term = '(bitcoin OR btc OR $BTC OR BTC OR BITCOIN OR $btc OR #btc OR #bitcoin) since:'+row['start_date']+' until:'+row['end_date']+' lang:en'
    print(naming + " in progress!")
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f'{term}').get_items()):  # declare a username
        e += 1
        print(f"Current scrape index: {e}", end="\r", flush=True)
        if i>max_result: #number of tweets you want to scrape
            break
            # print(movie_name, i, tweet)
        try:
            tweets_list.append([tweet.id, tweet.date, tweet.rawContent, tweet.user.id, tweet.user.username, tweet.user.displayname, tweet.user.created, tweet.user.followersCount, tweet.user.verified, tweet.user.location,  tweet.likeCount, tweet.retweetCount, tweet.bookmarkCount, tweet.quoteCount, tweet.pinned, tweet.hashtags, tweet.cashtags,  tweet.mentionedUsers, tweet.lang, tweet.coordinates, tweet.quotedTweet, tweet.retweetedTweet]) 
        except:
            continue
        
        
    tweets_df = pd.DataFrame(tweets_list, columns=['tweet_id', 'datetime', 'text', 'user_id', 'screen_name', 'full_name', 'user_created', 'followers_count', 'user_verified', 'user_location', 'likes', 'retweet_count', 'bookmark_count', 'quote_count', 'pinned', 'hashtags', 'cashtags', 'mentions', 'language', 'coordinates', 'quoted_tweet_id', 'retweeted_tweet_id'])

In [18]:
tweets_df.head(5)

,tweet_id,datetime,text,user_id,screen_name,full_name,user_created,followers_count,user_verified,user_location,...,bookmark_count,quote_count,pinned,hashtags,cashtags,mentions,language,coordinates,quoted_tweet_id,retweeted_tweet_id
0,1664059177000370178,2023-05-31 23:59:59+00:00,Apple Pay sauce still smacking \njuss tap in \...,235217038,bowers33uga,Lurdhacker1,2011-01-07 17:01:34+00:00,240,False,"Fort Worth, TX",...,0,0,None,None,None,None,en,None,None,None
1,1664059168595013634,2023-05-31 23:59:57+00:00,Bitcoin price index https://t.co/o7UcHJUhC6 #U...,2395138046,WorldCoinIndex,WorldCoinIndex,2014-03-17 21:39:08+00:00,12764,False,,...,0,0,None,"[USD, EUR, CNY, GBP, RUB]",None,None,en,None,None,None
2,1664059145828311040,2023-05-31 23:59:51+00:00,@CryptoCurrent77 @CDemanincor Just go away alr...,1464714519205273601,JeiOv081013,OvCryptoBagMan,2021-11-27 21:56:06+00:00,25,False,,...,0,0,None,None,None,"[https://twitter.com/CryptoCurrent77, https://...",en,None,None,None
3,1664059140388339712,2023-05-31 23:59:50+00:00,@CryptoDonAlt @gainzy222 @loomdart Here's the ...,615292671,ghanberly,Princess Freda,2012-06-22 14:26:13+00:00,79,False,Georgia,...,0,0,None,"[cryptocurrency, BTC, ETH]",None,"[https://twitter.com/CryptoDonAlt, https://twi...",en,None,None,None
4,1664059137251000320,2023-05-31 23:59:49+00:00,In time no one will stand against #Bitcoin htt...,16160335,CraigShipp,Craig Shipp,2008-09-06 20:19:25+00:00,9011,False,"Sarasota, Florida, USA",...,0,0,None,[Bitcoin],None,None,en,None,https://twitter.com/Dennis_Porter_/status/1664...,None


### Versión Alfa - App de extracción Tweets (v. 1.0.2)

In [3]:
class TweetScraper:
    def __init__(self, start_date, end_date, max_result, num_threads):
        self.start_date = start_date
        self.end_date = end_date
        self.max_result = max_result
        self.num_threads = num_threads
        self.tweets_list = []

    def clean_tweet(self, tweet):
        tweet = tweet.lower()
        tweet = re.sub(r'@[^\s]+', '', tweet)
        tweet = re.sub(r'(?<!\w)#(?!bitcoin|btc\b)\w+(?:,)?', '', tweet)
        tweet = re.sub(r'(?<!\w)\$(?!bitcoin|btc\b)\w+(?:,)?', '', tweet)
        tweet = re.sub(r'http[^\s,]+', '', tweet)
        tweet = re.sub(r'[^\w\s,$#]', '', tweet)
        tweet = re.sub(r'\s+', ' ', tweet)
        tweet = re.sub(r'^\s+|\s+$', '', tweet)

        return tweet

        
    def scrape_tweets(self, start, end):
        sub_tweets_list = []
        tweets_ids_csv = pd.read_csv('./dataset/tweets_ids.csv')
        df_tweets_ids = pd.DataFrame(tweets_ids_csv)
        list_tweets_ids = list(df_tweets_ids['tweet_id'])  # Obtener la lista de IDs existentes
        term = '(bitcoin OR btc OR $BTC OR BTC OR BITCOIN OR $btc OR #btc OR #bitcoin) since:{} until:{} lang:en'.format(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))
        print("{} in progress!".format(start.strftime('%Y-%m-%d')))
        
        existing_tweet_ids = set(list_tweets_ids)  # Convertir la lista de IDs existentes en un conjunto
        
        for i, tweet in enumerate(sntwitter.TwitterSearchScraper(term).get_items()):
            if i > self.max_result:
                break
            if tweet.id in existing_tweet_ids:
                continue
            try:
                cleaned_tweet = self.clean_tweet(tweet.rawContent)
                sub_tweets_list.append([tweet.id, tweet.date, cleaned_tweet, tweet.user.id, tweet.user.username, tweet.user.displayname, tweet.user.created, tweet.user.followersCount, tweet.user.verified, tweet.user.location, tweet.likeCount, tweet.retweetCount, tweet.bookmarkCount, tweet.quoteCount, tweet.hashtags, tweet.cashtags, tweet.mentionedUsers, tweet.lang, tweet.coordinates, tweet.quotedTweet])
                existing_tweet_ids.add(tweet.id)  # Agregar el ID del tweet a la lista de IDs existentes
                
            except:
                continue
        tweets_ids = pd.DataFrame({'tweet_id': list(existing_tweet_ids)})
        tweets_ids.to_csv('./dataset/tweets_ids.csv', index=False)
        return sub_tweets_list
    
    def scrape_parallel(self):
        date_range = pd.date_range(start=self.start_date, end=self.end_date, freq='1D')

        files_csv = glob.glob('./dataset/*.csv')
        filename_list_csv = [os.path.basename(file) for file in files_csv]
        
        output_filename_base = f'tweets_{self.start_date.strftime("%Y-%m-%d")}_{self.end_date.strftime("%Y-%m-%d")}'
        output_filename = output_filename_base + '.csv'

        # Verificar si el nombre de la variable existe en la lista de archivos
        if output_filename in filename_list_csv:
            # Extraer el número incremental del archivo existente
            incremental_number = 1
            while f"{output_filename_base}_{incremental_number}.csv" in filename_list_csv:
                incremental_number += 1
            
            # Construir el nuevo nombre de archivo con el número incremental
            new_output_filename = f"{output_filename_base}_{incremental_number}.csv"
        else:
            # El nombre de la variable no existe en la lista de archivos
            new_output_filename = output_filename_base + ".csv"
    
                
        with ThreadPoolExecutor(max_workers=self.num_threads) as executor:
            futures = []
            for i in range(len(date_range) - 1):
                start = date_range[i]
                end = date_range[i+1]
                futures.append(executor.submit(self.scrape_tweets, start, end))
            
            for future in futures:
                sub_tweets_list = future.result()
                self.tweets_list.extend(sub_tweets_list)
        
        tweets2_df = pd.DataFrame(self.tweets_list, columns=['tweet_id', 'datetime', 'text', 'user_id', 'screen_name', 'full_name', 'user_created', 'followers_count', 'user_verified', 'user_location', 'likes', 'retweet_count', 'bookmark_count', 'quote_count', 'hashtags', 'cashtags', 'mentions', 'language', 'coodinates', 'quoted_tweet_id'])
        tweets2_df.to_csv('./dataset/' + new_output_filename, index=False, sep='|')
        print(f"Output file generated: {new_output_filename}")


# Decorador para medir el tiempo de ejecución
def measure_execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = datetime.now()
        func(*args, **kwargs)
        end_time = datetime.now()
        execution_time = end_time - start_time
        print("Execution time: {}".format(execution_time))
    return wrapper


@measure_execution_time
def main():
    start_date = datetime(2023, 4, 25)
    end_date = datetime(2023, 4, 27)
    max_result = 50000
    num_threads = 10
    
    scraper = TweetScraper(start_date, end_date, max_result, num_threads)
    scraper.scrape_parallel()

if __name__ == "__main__":
    main()

2023-04-25 in progress!
2023-04-26 in progress!


Skipping unrecognised entry ID: 'promoted-tweet-1651254816876666882-25d61666dcfb76b1'
Skipping unrecognised entry ID: 'promoted-tweet-1665739194180816901-25d6ccc19ed04850'
Skipping unrecognised entry ID: 'promoted-tweet-1667838584848777216-25d6c8b4f58a5b01'
Skipping unrecognised entry ID: 'promoted-tweet-1651254816876666882-25d6dbbb0b58aae6'
Skipping unrecognised entry ID: 'promoted-tweet-1665739194180816901-25d6e3ea29763d0e'
Skipping unrecognised entry ID: 'promoted-tweet-1667838584848777216-25d635f9129a34d3'
Skipping unrecognised entry ID: 'promoted-tweet-1651254816876666882-25d6dde5dd742f3c'
Skipping unrecognised entry ID: 'promoted-tweet-1651254816876666882-25d6bf069d4e042f'
Skipping unrecognised entry ID: 'promoted-tweet-1665739194180816901-25d6d935bf069516'
Skipping unrecognised entry ID: 'promoted-tweet-1665739194180816901-25d6849e492c0673'
Skipping unrecognised entry ID: 'promoted-tweet-1651254816876666882-25d6ab349a9d4a78'
Unavailable user in card on tweet 1651013036302376964


Output file generated: tweets_2023-04-25_2023-04-27.csv
Execution time: 0:27:55.988805


In [4]:
data_csv = pd.read_csv('./dataset/tweets_2023-04-25_2023-04-27.csv', sep='|')
df_csv4 = pd.DataFrame(data_csv)
df_csv4.info()
df_csv4.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38254 entries, 0 to 38253
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tweet_id         38254 non-null  int64 
 1   datetime         38254 non-null  object
 2   text             38254 non-null  object
 3   user_id          38254 non-null  int64 
 4   screen_name      38254 non-null  object
 5   full_name        38253 non-null  object
 6   user_created     38254 non-null  object
 7   followers_count  38254 non-null  int64 
 8   user_verified    38254 non-null  bool  
 9   user_location    21387 non-null  object
 10  likes            38254 non-null  int64 
 11  retweet_count    38254 non-null  int64 
 12  bookmark_count   38254 non-null  int64 
 13  quote_count      38254 non-null  int64 
 14  hashtags         17876 non-null  object
 15  cashtags         7109 non-null   object
 16  mentions         15781 non-null  object
 17  language         38254 non-null

,tweet_id,datetime,text,user_id,screen_name,full_name,user_created,followers_count,user_verified,user_location,likes,retweet_count,bookmark_count,quote_count,hashtags,cashtags,mentions,language,coodinates,quoted_tweet_id
0,1651013206486179843,2023-04-25 23:59:57+00:00,etherium was the bitcoin killer nope shiba inu...,1594557859383201794,JokesMccracken,WLG WLG,2022-11-21 05:07:00+00:00,21,False,NaN,3,0,0,0,NaN,NaN,"[User(username='SawyerMerritt', id=243013409, ...",en,NaN,NaN
1,1651013192326193155,2023-04-25 23:59:54+00:00,i think first republic basically failing is an...,1487520532962725890,BryanRu44425584,Bryan Russell,2022-01-29 20:18:57+00:00,26,False,NaN,3,0,0,0,NaN,NaN,"[User(username='cryptorecruitr', id=37881270, ...",en,NaN,NaN
2,1651013191575523335,2023-04-25 23:59:53+00:00,is life you have to be careful with the invest...,1505359960259014656,ramonmo62904793,Oliver_MT,2022-03-20 01:47:12+00:00,91,False,Brazil,1,0,0,0,NaN,NaN,"[User(username='elserrano87', id=1308102018, d...",en,NaN,NaN
3,1651013188476112896,2023-04-25 23:59:53+00:00,thank you guys for making fat people eat less,1285782737995980800,Bitcoin_ish,justwins.eth,2020-07-22 03:45:25+00:00,49,False,NaN,2,0,0,0,NaN,NaN,"[User(username='PeterSchiff', id=56562803, dis...",en,NaN,NaN
4,1651013185057566723,2023-04-25 23:59:52+00:00,you have never worked a day in your life made ...,1313955205,ccameletti,Carol Cameletti,2013-03-29 14:10:11+00:00,112,False,NaN,1,0,0,0,NaN,NaN,"[User(username='PierrePoilievre', id=242827267...",en,NaN,NaN
5,1651013175683461120,2023-04-25 23:59:50+00:00,$btc update,1315513208380284933,Everesttrade12,MT EVEREST,2020-10-12 04:43:03+00:00,122,False,Worldwide,0,0,0,0,NaN,['BTC'],NaN,en,NaN,NaN
6,1651013161401696256,2023-04-25 23:59:46+00:00,bitcoin price index,2395138046,WorldCoinIndex,WorldCoinIndex,2014-03-17 21:39:08+00:00,12769,False,NaN,0,0,0,0,"['USD', 'EUR', 'CNY', 'GBP', 'RUB']",NaN,NaN,en,NaN,NaN
7,1651013149523607552,2023-04-25 23:59:43+00:00,old woman yells at #bitcoin,1522311095360245761,iambenjammin,₿en Jammin,2022-05-05 20:23:59+00:00,180,False,NaN,1,0,0,0,['bitcoin'],NaN,"[User(username='BitcoinNewsCom', id=3005014565...",en,NaN,NaN
8,1651013141604749313,2023-04-25 23:59:41+00:00,ccie found #bitcoin in a blue vault at this lo...,1281786980,vitapin,Vitaliy P,2013-03-19 23:27:10+00:00,43,False,"Vancouver, British Columbia",2,0,0,0,"['bitcoin', 'coinhuntworld', 'cryptocurrency']",NaN,NaN,en,NaN,NaN
9,1651013138568065025,2023-04-25 23:59:41+00:00,ordered a poster from and 3 weeks later nothin...,1527534253046497280,Webster4587,Webster,2022-05-20 06:19:05+00:00,61,False,NaN,1,0,0,0,NaN,NaN,"[User(username='BitcoinMagazine', id=361289499...",en,NaN,NaN
